In [2]:
import cv2
import numpy as np
from frame_helper import generate_frames, extractFeatures, bfmatcher
import torch

vid_path = '5.hevc'
kps_1 = None
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, _verbose=False)
model.classes = [2,3,4,6,7,8]
model.conf = 0.2
model.iou = 0.45

for i, (p, c) in enumerate(generate_frames(vid_path)):
  bb = model(p).xyxy[0].numpy()
  bb = bb.astype(int)

  for b in bb:
    cv2.rectangle(c, (b[0],b[1]), (b[2],b[3]), (100,0,200), 2)
 
  if kps_1 is not None:
    kps_1, des_1 = kps_2, des_2
  else:
    kps_1, des_1 = extractFeatures(p)
  kps_2, des_2 = extractFeatures(c)


  good_kps = np.ones((len(kps_2)))
  for i, kp in enumerate(kps_2):
    for b in bb:
      if b[0]<kp[0]<b[2] and b[1]<kp[1]<b[3]:
        good_kps[i] = 0
        break

  kps_2 = kps_2[good_kps.astype(bool)]
  des_2 = des_2[good_kps.astype(bool)]

  p1, p2 = bfmatcher([kps_1, kps_2], [des_1, des_2], threshold=0.001)
  
  for p in p2:
    cv2.circle(c, (int(p[0]), int(p[1])), 1, (255,255,0))  
     
  for k1, k2 in zip(p1, p2):
    cv2.line(c, tuple(k1.astype(int)), tuple(k2.astype(int)), (0,200,200), 1)
  cv2.imshow('v', c)
  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()

Using cache found in C:\Users\Sayaz/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-6 Python-3.9.7 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
